In [ ]:
import pkg_resources
# Install packages
package_names = ['keras_vggface', 'keras_applications', 'batch_face', 'speechbrain']
for package_name in package_names:
  try:
    dist = pkg_resources.get_distribution(package_name)
    print(f"{package_name} is already installed with version {dist.version}")
  except pkg_resources.DistributionNotFound:
    print(f"{package_name} is not installed, installing now...")
    !pip install {package_name}

# !pip install --upgrade --force-reinstall keras_vggface
# !pip install keras_vggface
# !pip install keras_applications
# !pip install batch_face

# Fix package files
!sed -i 's/from keras.utils import layer_utils/from tensorflow.python.keras.utils import layer_utils/' /usr/local/lib/python3.10/dist-packages/keras_vggface/models.py
!sed -i 's/from keras.utils.data_utils import get_file/from tensorflow.python.keras.utils.data_utils import get_file/' /usr/local/lib/python3.10/dist-packages/keras_vggface/models.py
!sed -i 's/from keras.utils.data_utils import get_file/from tensorflow.python.keras.utils.data_utils import get_file/' /usr/local/lib/python3.10/dist-packages/keras_vggface/utils.py
!sed -i 's/from keras.engine.topology import get_source_inputs/from tensorflow.python.keras.utils.layer_utils import get_source_inputs/' /usr/local/lib/python3.10/dist-packages/keras_vggface/models.py

# import argparse
import numpy as np
import pandas as pd
import os
import time
from scipy import stats
import pickle
import sys
# import speechbrain
import multiprocessing as mp
# from select_video_subset import select_video_subset

import warnings
warnings.filterwarnings('ignore', category = FutureWarning)

# Mount drive
from google.colab import drive

def is_drive_mounted():
    drive_path = '/content/drive'
    return os.path.isdir(drive_path) and os.listdir(drive_path)

# Mount Google Drive if it is not already mounted
if not is_drive_mounted():
    drive.mount('/content/drive')
    print("Google Drive is mounted now!")
else:
    print("Google Drive is already mounted.")

expected_directory = '/content/notebook_modules'

# Check if the expected directory or file exists
if not os.path.exists(expected_directory):
    print("Modules directory does not exist, extracting the ZIP file...")
    # Extract the ZIP file since the directory doesn't exist
    # !unzip '/content/notebook_modules.zip' -d /content/
    !unzip '/content/drive/MyDrive/Thesis_Data/CREMA_runs/import_notebook_modules/Win_10_step_5/notebook_modules.zip' -d /content/
else:
    print("Modules directory already exists. No need to extract the ZIP file.")

sys.path.append('/content/notebook_modules')

import sequences
import get_face_areas
from get_models import load_weights_EE, load_weights_LSTM
import run_functions

multimodal_modules_directory = '/content/multimodal_modules'

# Check if the expected directory or file exists
if not os.path.exists(multimodal_modules_directory):
    print("Multimodal modules directory does not exist, extracting the ZIP file...")
    # Extract the ZIP file since the directory doesn't exist
    # !unzip '/content/notebook_modules.zip' -d /content/
    !unzip '/content/drive/MyDrive/Thesis_Data/chinese/multimodal_modules.zip' -d /content/
else:
    print("Multimodal modules directory already exists. No need to extract the ZIP file.")

sys.path.append('/content/multimodal_modules')

from audio_model import AudioModel
from visual_model import VisualModel

# Data folders
video_data_folder = '/content/VideoMP4_testing'
audio_data_folder = '/content/AudioWAV_resampled'

# Extract zip folder on Google Drive and save it in Google Colab workspace
if not os.path.exists(video_data_folder):
    print("Video data folder does not exist, extracting the ZIP file...")
    # Extract the ZIP file since the directory doesn't exist
    !unzip '/content/drive/MyDrive/Thesis_Data/chinese/video_files.zip' -d /content/
else:
    print("Video data folder already exists. No need to extract the ZIP file.")

if not os.path.exists(audio_data_folder):
    print("Audio data folder does not exist, extracting the ZIP file...")
    # Extract the ZIP file since the directory doesn't exist
    !unzip '/content/drive/MyDrive/Thesis_Data/chinese/audio_files.zip' -d /content/
else:
    print("Audio data folder already exists. No need to extract the ZIP file.")

def get_dataset(dataset, directory):
    # Define the dataset and the directory
    if dataset == "CREMA-D":
        data = pd.read_csv(os.path.join(directory,r"CREMA-D\labels_testing.csv"))
        directory = os.path.join(directory,r"CREMA-D\audio_testing")
        my_encoding_dict_dataset = {'NEU': 0, 'ANG': 1, 'HAP': 2, 'SAD': 3}

    elif dataset == "CREMA-D-voted":
        data = pd.read_csv(os.path.join(directory,r"CREMA-D\labels_v_testing.csv"))
        directory = os.path.join(directory,r"CREMA-D\audio_v_testing")
        my_encoding_dict_dataset = {'N': 0, 'A': 1, 'H': 2, 'S': 3}

    files = []

    # Get a list of all files in the directory (audio files, change for video files)
    for file in os.listdir(directory):
        if file.endswith('.wav'):
            files.append(file)

    return files, data, directory, my_encoding_dict_dataset

def get_crema_d_dataset(audio_directory, video_directory, labels_file, voted=True):
    # Define the dataset and the directory
    if voted:
        # data = pd.read_csv(os.path.join(directory, labels_file))
        # directory = os.path.join(directory,r"CREMA-D\audio_testing")
        my_encoding_dict_dataset = {'N': 0, 'A': 1, 'H': 2, 'S': 3}
    else:
        # data = pd.read_csv(os.path.join(directory,r"CREMA-D\labels_v_testing.csv"))
        # directory = os.path.join(directory,r"CREMA-D\audio_v_testing")
        my_encoding_dict_dataset = {'NEU': 0, 'ANG': 1, 'HAP': 2, 'SAD': 3}

    audio_files = []
    video_files = []

    # Get a list of all files in the directory (audio files, change for video files)
    for file in os.listdir(audio_directory):
        if file.endswith('.wav'):
            audio_files.append(file)

    for file in os.listdir(video_directory):
        if file.endswith('.mp4'):
            video_files.append(file)

    return audio_files, video_files, my_encoding_dict_dataset


# Debugging function to check only one file
def get_single_file(file):
    # Get a single file for debugging
    data = pd.read_csv(r"C:\Users\DANIEL\Desktop\thesis\CREMA-D\labels_testing.csv")
    directory = r"C:\Users\DANIEL\Desktop\thesis\CREMA-D\audio_testing"
    for row in data.iterrows():
        if row[1]['File'] == file:
            data = row[1]
            break
    my_encoding_dict_dataset = {'NEU': 0, 'ANG': 1, 'HAP': 2, 'SAD': 3}
    return file, data, directory, my_encoding_dict_dataset

def get_label_keys(data, my_encoding_dict_dataset):
    # Get the label keys from the dataset
    true_labels_multi = data['Emotion'] # Change this to the true labels of the multimodal model
    true_labels_audio = data['Emotion'] # Change this to the true labels of the audio model
    true_labels_visual = data['Emotion'] # Change this to the true labels of the visual model
    label_keys_multi = true_labels_multi.map(my_encoding_dict_dataset).values
    label_keys_audio = true_labels_audio.map(my_encoding_dict_dataset).values
    label_keys_visual = true_labels_visual.map(my_encoding_dict_dataset).values

    return label_keys_multi, true_labels_multi

# Separete the audio and the video of the file
def separate_audio_video(file):
    # Separate the audio and video files

    audio_directory = r"C:\Users\DANIEL\Desktop\thesis\CREMA-D\AudioWav"
    video_directory = r"C:\Users\DANIEL\Desktop\thesis\CREMA-D\VideoFlash"

    # Lood for audio file in AudioWav folder
    audio_files = (os.path.join(audio_directory, file)+'.wav')

    # Look for video file in VideoFlash folder
    video_files = (os.path.join(video_directory, file)+'.flv')

    return audio_files, video_files

# Call both models to classify the audio and the video
def save_results(results, name):
    # Save the results for debugging as a csv file
    results_df = pd.DataFrame(results)
    # Get current directory
    current_dir = os.getcwd()
    results_save_path = os.path.join(current_dir, 'multimodal_results')
    results_df.to_csv(os.path.join(results_save_path, f'{name}.csv'), index=False)
    # results_df.to_csv(f'{name}.csv', index=False)

def select_final_label(final_probabilities):
    # Select the final label based on the combined probabilities
    final_label = np.argmax(final_probabilities)
    return final_label

def combine_probabilities(audio_prob, video_prob, audio_weight=0.4, video_weight=0.6):
    # Implement your logic to combine probabilities from both models
    # print()
    # print()
    # print("-----------------Combining Probabilities-----------------")
    # print()
    # print("Audio Probabilities: ", audio_prob)
    audio_weighted = audio_prob*audio_weight
    # print("Audio Prob. weighted: ", audio_weighted)
    # print()
    # print("Video Probabilities: ", video_prob)
    video_weighted = video_prob*video_weight
    # print("Video Prob. weighted: ", video_weighted)
    combined_prob = audio_weighted + video_weighted
    # print()
    # print("Combined Probabilities: ", combined_prob)
    return combined_prob

def process_audio(audio_file):
    audio_classifier = AudioModel()
    out_prob, score, index, text_lab = audio_classifier.classify_audio_file(audio_file)
    out_prob = out_prob.numpy()
    # Add three zeros to match the dimensions of the visual model
    out_prob = np.append(out_prob, [0, 0, 0]).reshape(1,7)
    return out_prob, score, index, text_lab

def process_video(video_file, backbone_model_path, LSTM_model_path):
    video_classifier = VisualModel()
    out_prob, score, index, text_lab = video_classifier.classify_video_file(video_file, backbone_model_path, LSTM_model_path)
    out_prob = reorder_video_probabilities(out_prob)
    return out_prob, score, index, text_lab

def reorder_video_probabilities(video_probabilities):
    # Reorder the video_probabilities to match the audio_probabilities
    # Order = ['neu', 'ang', 'hap', 'sad']
    # video_model = ['Neutral', 'Happiness', 'Sadness', 'Surprise', 'Fear', 'Disgust', 'Anger']
    video_model_dict = {'neu' : 0, 'hap' : 1, 'sad' : 2, 'sur' : 3, 'fea' : 4, 'dis' : 5, 'ang' : 6}

    new_probabilities = np.array([video_probabilities[video_model_dict['neu']],
                                  video_probabilities[video_model_dict['ang']],
                                  video_probabilities[video_model_dict['hap']],
                                  video_probabilities[video_model_dict['sad']],
                                  video_probabilities[video_model_dict['sur']],
                                  video_probabilities[video_model_dict['fea']],
                                  video_probabilities[video_model_dict['dis']]])

    new_probabilities = new_probabilities.reshape(1,7)
    # compare old and new probabilities
    # print("Old Probabilities: ")
    # print(video_probabilities)
    # print("New Probabilities: ")
    # print(new_probabilities)
    return new_probabilities

# Call both models to classify the audio and the video
if __name__ == '__main__':
    # Test runs are saved in Google Drive in folders named Run_1, Run_2, etc.
    # Check the number of the latest run folder
    run_folders = [folder for folder in os.listdir('/content/drive/MyDrive/Thesis_Data/chinese') if 'Run_' in folder]

    continue_from_checkpoint = False
    backbone_model_path = '/content/drive/MyDrive/Thesis_Data/CREMA_runs/Pretrained_models/Backbone_models/weights_0_66_37_wo_gl.h5'
    LSTM_model_path = '/content/drive/MyDrive/Thesis_Data/CREMA_runs/Pretrained_models/LSTM_models/CREMA-D_with_config.h5'

    # If run folders is empty, this is the first run
    # Check if run folders is empty
    if not run_folders:
        this_run = 1
        this_run_folder = f'/content/drive/MyDrive/Thesis_Data/chinese/Run_{this_run}'
        os.makedirs(this_run_folder)
        # Create subfolder for checkpoints
        checkpoint_folder = os.path.join(this_run_folder, 'checkpoints')
        os.makedirs(checkpoint_folder)
    elif continue_from_checkpoint:
        latest_run = max([int(folder.split('_')[1]) for folder in run_folders])
        this_run = latest_run
        this_run_folder = f'/content/drive/MyDrive/Thesis_Data/chinese/Run_{this_run}'
        checkpoint_folder = os.path.join(this_run_folder, 'checkpoints')
        checkpoint_files = [csv_file for csv_file in os.listdir(checkpoint_folder)]
        last_checkpoint = max([int(checkpoint_file.split("_")[-1].split(".")[0]) for checkpoint_file in checkpoint_files])
        print(f"Continuing from checkpoint {last_checkpoint}. (Run {this_run})")
    else:
        latest_run = max([int(folder.split('_')[1]) for folder in run_folders])
        this_run = latest_run + 1
        this_run_folder = f'/content/drive/MyDrive/Thesis_Data/chinese/Run_{this_run}'
        os.makedirs(this_run_folder)
        # Create subfolder for checkpoints
        checkpoint_folder = os.path.join(this_run_folder, 'checkpoints')
        os.makedirs(checkpoint_folder)

    # label_model = ['Neutral', 'Happiness', 'Sadness', 'Surprise', 'Fear', 'Disgust', 'Anger']
    # my_encoding_dict_model = {'neu': 0, 'ang': 1, 'hap': 2, 'sad': 3} # Change this to include (or not) the extra classes of the visual model
    my_encoding_dict_model = {'neu': 0, 'ang': 1, 'hap': 2, 'sad': 3, 'sur': 4, 'fea': 5, 'dis': 6}
    label_names = ['neu', 'ang', 'hap', 'sad'] # Same as above

    label_model_decoder = {0: 'Neutral', 1: 'Anger', 2: 'Happiness', 3: 'Sadness', 4: 'Surprise', 5: 'Fear', 6: 'Disgust'}

    # dataset = "CREMA-D" # Change this to the dataset you are using
    # directory = "path_to_datasets" # Change this to the directory where you save the datasets
    # file = '1001_DFA_ANG_XX' # Change this to the file you want to classify
    # file = '1001_IEO_SAD_MD'

    # audio_folder = r"C:\MyDocs\DTU\MSc\Thesis\Data\CREMA-D\CREMA-D\AudioWAV"
    # audio_folder = r"C:\_HomeDocs\Ari\DTU\00-MSc\Thesis\Data\AudioWAV_testing"
    # video_folder = r"C:\MyDocs\DTU\MSc\Thesis\Data\CREMA-D\CREMA-D\VideoMP4"
    # video_folder = r"C:\_HomeDocs\Ari\DTU\00-MSc\Thesis\Data\VideoMP4_testing"
    audio_folder = audio_data_folder
    video_folder = video_data_folder

    # files, data, directory, my_encoding_dict_dataset = get_dataset(dataset, directory)
    labels_data = pd.read_csv(r"/content/drive/MyDrive/Thesis_Data/chinese/label_corrected_testing.csv")
    filenames = labels_data['filename'].tolist()
    emotions_list = labels_data['Emotion'].tolist()
    # labels_list = labels_data['Label'].tolist()
    total_data_length = len(filenames)

    # predictions_df = labels.copy()
    # predictions_df['audio_prob'] = None
    # predictions_df['video_prob'] = None

    if continue_from_checkpoint:
        filenames = filenames[last_checkpoint:]
        emotions_list = emotions_list[last_checkpoint:]
        progress = last_checkpoint + 1
    else:
        progress = 1

    predictions_df = pd.DataFrame(columns=['filename', 'Emotion', 'audio_prob', 'video_prob', 'checkpoint'])
    predictions_df['filename'] = filenames
    predictions_df['Emotion'] = emotions_list
    # predictions_df['Label'] = labels_list

    # files, data, directory, my_encoding_dict_dataset = get_single_file(file)
    # label_keys, true_labels = get_label_keys(data, my_encoding_dict_dataset)
    # audio_input, video_input = separate_audio_video(file)
    # debug_counter = 0
    # audio_probs_list = []
    # audio_probs_list = []
    checkpoint_row = 1
    # mp.set_start_method('spawn')
    # with mp.Pool(2) as pool:
    for file in filenames:
        print("-----------------------------------------------")
        print(f"Processing file {progress}/{total_data_length}")
        print("-----------------------------------------------")
        # if debug_counter>=4:
            # break
        audio_input = os.path.join(audio_folder, file + '.wav')
        video_input = os.path.join(video_folder, file + '.mp4')
        # audio_input = file
        print(f'Audio Input: {audio_input}')
        print(f'Video Input: {video_input}')
        if file == "1076_MTI_NEU_XX":
            print("Corrupted file. Skipping...")
            progress += 1
            checkpoint_row += 1
            continue
        if file == "1076_MTI_SAD_XX":
            print("Corrupted file. Skipping...")
            progress += 1
            checkpoint_row += 1
            continue

        # audio_result = pool.apply_async(process_audio, (audio_input,))
        # video_result = pool.apply_async(process_video, (video_input, backbone_model_path, LSTM_model_path))
        audio_result = process_audio(audio_input)
        video_result = process_video(video_input, backbone_model_path, LSTM_model_path)

        # audio_probabilities, _, _, _ = audio_result.get()
        # video_probabilities, _, _, _ = video_result.get()
        audio_probabilities = audio_result[0]
        video_probabilities = video_result[0]

        # Save separate results for debugging
        # save_results(audio_probabilities, 'audio_results')
        # save_results(video_probabilities, 'video_results')

        # print("We have now reordered the video probabilities")
        # print(video_probabilities.shape)
        # print(video_probabilities)

        # Combine results and determine final label
        # final_probabilities = combine_probabilities(audio_probabilities, video_probabilities, audio_weight=0.5, video_weight=0.5)
        # final_label = select_final_label(final_probabilities)
        # final_label_name = label_model_decoder[final_label]

        # print(f'Final Label: {final_label}', f'Final Probabilities: {final_probabilities}')
        # print(f'Final Label Name: {final_label_name}')
        # print(f'True Label: {true_labels}')
        # audio_probs_list.append
        # video_probs_list
        # predictions_df.loc[predictions_df['filename'] == file, 'audio_prob'] = audio_probabilities
        # predictions_df.loc[predictions_df['filename'] == file, 'video_prob'] = video_probabilities
        predictions_df.at[predictions_df[predictions_df['filename'] == file].index[0], 'audio_prob'] = audio_probabilities
        predictions_df.at[predictions_df[predictions_df['filename'] == file].index[0], 'video_prob'] = video_probabilities
        predictions_df.at[predictions_df[predictions_df['filename'] == file].index[0], 'checkpoint'] = progress

        # debug_counter += 1
        if progress % 25 == 0:
            # save the DataFrame to a csv file
            checkpoint_data = predictions_df.head(checkpoint_row)
            checkpoint_data.to_csv(os.path.join(checkpoint_folder,f'run_{this_run}_predicted_checkpoint_{progress}.csv'), index=False)
            checkpoint_data.to_pickle(os.path.join(checkpoint_folder,f'run_{this_run}_predicted_checkpoint_{progress}.pkl'))
            print(f"Checkpoint {progress} saved.")
        progress += 1
        checkpoint_row += 1

    print()
    print()
    print("------------------------------------------------------------------------------")
    # save the DataFrame to a csv file
    predictions_df.to_csv(os.path.join(this_run_folder,f'run_{this_run}_predicted.csv'), index=False)
    predictions_df.to_pickle(os.path.join(this_run_folder,f'run_{this_run}_predicted.pkl'))
    predictions_df.head()

keras_vggface is already installed with version 0.6
keras_applications is already installed with version 1.0.8
batch_face is already installed with version 1.4.0
speechbrain is already installed with version 1.0.0
Google Drive is already mounted.
Modules directory already exists. No need to extract the ZIP file.
Multimodal modules directory already exists. No need to extract the ZIP file.
Video data folder already exists. No need to extract the ZIP file.
Audio data folder already exists. No need to extract the ZIP file.
-----------------------------------------------
Processing file 1/600
-----------------------------------------------
Audio Input: /content/AudioWAV_resampled/video_0001_0004.wav
Video Input: /content/VideoMP4_testing/video_0001_0004.mp4


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Name video:  video_0001_0004.mp4
Number total of frames:  97
FPS:  25.0
Video duration: 3.88 s
Frame width: 1920
Frame height: 804
-----------------------------------------------
Processing file 2/600
-----------------------------------------------
Audio Input: /content/AudioWAV_resampled/video_0001_0010.wav
Video Input: /content/VideoMP4_testing/video_0001_0010.mp4


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Name video:  video_0001_0010.mp4
Number total of frames:  82
FPS:  25.0
Video duration: 3.28 s
Frame width: 1920
Frame height: 804
-----------------------------------------------
Processing file 3/600
-----------------------------------------------
Audio Input: /content/AudioWAV_resampled/video_0001_0013.wav
Video Input: /content/VideoMP4_testing/video_0001_0013.mp4


...

Name video:  video_0060_0036.mp4
Number total of frames:  72
FPS:  30.0
Video duration: 2.4 s
Frame width: 1280
Frame height: 540
-----------------------------------------------
Processing file 599/600
-----------------------------------------------
Audio Input: /content/AudioWAV_resampled/video_0060_0040.wav
Video Input: /content/VideoMP4_testing/video_0060_0040.mp4


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Name video:  video_0060_0040.mp4
Number total of frames:  56
FPS:  30.0
Video duration: 1.87 s
Frame width: 1280
Frame height: 540
-----------------------------------------------
Processing file 600/600
-----------------------------------------------
Audio Input: /content/AudioWAV_resampled/video_0060_0042.wav
Video Input: /content/VideoMP4_testing/video_0060_0042.mp4


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Name video:  video_0060_0042.mp4
Number total of frames:  61
FPS:  30.0
Video duration: 2.03 s
Frame width: 1280
Frame height: 540
Checkpoint 600 saved.


------------------------------------------------------------------------------
